# Organizar arquivos NC

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
!pip install netCDF4 h5netcdf




In [ ]:
import xarray as xr
import geopandas as gpd
import rioxarray
import pandas as pd
import os
from tqdm import tqdm  # Apenas para visualizar o progresso (opcional)

# === INSIRA SEUS CAMINHOS AQUI ===
pasta_nc = r"E:\CLIMBRA\Prec"  # Pasta onde estão os .nc
caminho_saida = r"E:\CLIMBRA\Extract\pr"  # Pasta onde salvar os CSVs
shapefile = r"C:\Users\Matheus Marinho\Desktop\IGUAÇU_OTTO\Shp\Buffer.shp"

# === Nome da variável no arquivo .nc ===
variavel = "pr"  # Exemplo: Umidade relativa

# === Abrir shapefile e garantir CRS correto ===
bacia = gpd.read_file(shapefile)
bacia = bacia.to_crs("EPSG:4326")

# === Listar todos os arquivos .nc da pasta ===
arquivos_nc = [os.path.join(pasta_nc, f) for f in os.listdir(pasta_nc) if f.endswith(".nc")]

# === Loop para processar cada arquivo ===
for caminho_nc in tqdm(arquivos_nc, desc="Processando arquivos"):

    try:
        # Abrir NetCDF e configurar para rioxarray
        ds = xr.open_dataset(caminho_nc)
        dados = ds[variavel]
        dados.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        dados.rio.write_crs("EPSG:4326", inplace=True)

        # Recorte espacial com base no shapefile
        dados_recorte = dados.rio.clip(bacia.geometry, bacia.crs, drop=True)

        # Converter para DataFrame e remover nulos
        df = dados_recorte.to_dataframe().reset_index()
        df = df.dropna(subset=[variavel])

        # Gerar nome de saída baseado no nome do .nc
        nome_arquivo = os.path.basename(caminho_nc).replace(".nc", ".csv")
        caminho_completo_saida = os.path.join(caminho_saida, nome_arquivo)

        # Salvar CSV
        df.to_csv(caminho_completo_saida, index=False)
        print(f"CSV salvo: {caminho_completo_saida}")

    except Exception as e:
        print(f"Erro ao processar {caminho_nc}: {e}")

In [ ]:
import xarray as xr
import geopandas as gpd
import rioxarray
import pandas as pd
import os
from tqdm import tqdm  # Apenas para visualizar o progresso (opcional)

# === INSIRA SEUS CAMINHOS AQUI ===
pasta_nc = r"E:\CLIMBRA\Prec"  # Pasta onde estão os .nc
caminho_saida = r"E:\CLIMBRA\Extract\pr"  # Pasta onde salvar os CSVs
shapefile = r"C:\Users\Matheus Marinho\Desktop\IGUAÇU_OTTO\Shp\Buffer.shp"

# === Nome da variável no arquivo .nc ===
variavel = "pr"  # Exemplo: Umidade relativa

# === Abrir shapefile e garantir CRS correto ===
import fiona
from shapely.geometry import shape
import geopandas as gpd

with fiona.open(shapefile, "r") as src:
    features = [shape(feat["geometry"]) for feat in src]
    bacia = gpd.GeoDataFrame(geometry=features, crs=src.crs)
bacia = bacia.to_crs("EPSG:4326")

# === Listar todos os arquivos .nc da pasta ===
arquivos_nc = [os.path.join(pasta_nc, f) for f in os.listdir(pasta_nc) if f.endswith(".nc")]

# === Loop para processar cada arquivo ===
for caminho_nc in tqdm(arquivos_nc, desc="Processando arquivos"):

    try:
        # Abrir NetCDF e configurar para rioxarray
        ds = xr.open_dataset(caminho_nc)
        dados = ds[variavel]
        dados.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        dados.rio.write_crs("EPSG:4326", inplace=True)

        # Recorte espacial com base no shapefile
        dados_recorte = dados.rio.clip(bacia.geometry, bacia.crs, drop=True)

        # Converter para DataFrame e remover nulos
        df = dados_recorte.to_dataframe().reset_index()
        df = df.dropna(subset=[variavel])

        # Gerar nome de saída baseado no nome do .nc
        nome_arquivo = os.path.basename(caminho_nc).replace(".nc", ".csv")
        caminho_completo_saida = os.path.join(caminho_saida, nome_arquivo)

        # Salvar CSV
        df.to_csv(caminho_completo_saida, index=False)
        print(f"CSV salvo: {caminho_completo_saida}")

    except Exception as e:
        print(f"Erro ao processar {caminho_nc}: {e}")

In [ ]:
import xarray as xr
import geopandas as gpd
import rioxarray
import pandas as pd
import os
from tqdm import tqdm
import fiona
from shapely.geometry import shape

# === CAMINHOS ===
pasta_nc = r"E:\CLIMBRA\Prec"
caminho_saida = r"E:\CLIMBRA\Extract\pr"
shapefile = r"C:\Users\Matheus Marinho\Desktop\IGUAÇU_OTTO\Shp\Buffer.shp"
variavel = "pr"

# === Abrir shapefile de forma segura com fiona ===
with fiona.open(shapefile, "r") as src:
    features = [shape(feat["geometry"]) for feat in src]
    bacia = gpd.GeoDataFrame(geometry=features, crs=src.crs)
bacia = bacia.to_crs("EPSG:4326")

# === Listar arquivos .nc ===
arquivos_nc = [os.path.join(pasta_nc, f) for f in os.listdir(pasta_nc) if f.endswith(".nc")]

# === Loop principal ===
for caminho_nc in tqdm(arquivos_nc, desc="Processando arquivos"):

    try:
        # Nome base do modelo
        nome_modelo = os.path.splitext(os.path.basename(caminho_nc))[0]

        # Criar subpasta para o modelo
        pasta_modelo = os.path.join(caminho_saida, nome_modelo)
        os.makedirs(pasta_modelo, exist_ok=True)

        # Abrir NetCDF e configurar rioxarray
        ds = xr.open_dataset(caminho_nc)
        dados = ds[variavel]
        dados.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        dados.rio.write_crs("EPSG:4326", inplace=True)

        # Recorte espacial
        dados_recorte = dados.rio.clip(bacia.geometry, bacia.crs, drop=True)

        # Converter para DataFrame e remover nulos
        df = dados_recorte.to_dataframe().reset_index()
        df = df.dropna(subset=[variavel])

        # Exportar um CSV por ponto
        for (lat, lon), grupo in df.groupby(["lat", "lon"]):
            nome_arquivo = f"{nome_modelo}_lat_{lat:.2f}_lon_{lon:.2f}.csv"
            caminho_csv = os.path.join(pasta_modelo, nome_arquivo)
            grupo[["time", variavel]].to_csv(caminho_csv, index=False)

    except Exception as e:
        print(f"Erro ao processar {caminho_nc}: {e}")

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import time

# Diretório de entrada
my_dir = r'G:/Meu Drive/2_MESTRADO/1_Dissertação/Dados/Clima/Intermediario'
print('Seu diretório é:', my_dir)
time.sleep(1)

# Listar arquivos CSV no diretório
files = glob.glob(my_dir + '/*.csv')

# Nome das colunas atualizadas
_base_columns = [
    'Day',
    'Month',
    'Year',
    'tmean_ens',
    'rh_ens',
    'wnd_ens',
    'srad_ens',
    'patm_ens'
]

# Tipo dos dados da coluna
_dtype = {
    'Day': int,
    'Month': int,
    'Year': int,
    'tmean_ens': float,
    'rh_ens': float,
    'wnd_ens': float,
    'srad_ens': float,
    'patm_ens': float
}

# Espaço nas colunas
col_space = [6, 5, 5, 11, 11, 11, 11, 11]

# Ler e processar os arquivos CSV
for file in files:
    df = pd.read_csv(file,
                     delimiter=';',  # Tipo de delimitador
                     names=_base_columns,  # Nome das colunas
                     dtype=_dtype
                     ).reset_index(drop=True)

    # Formatar as colunas numéricas para terem apenas uma casa decimal
    for col in _base_columns[3:]:
        df[col] = df[col].apply('{:,.1f}'.format)

    # Adicione a coluna 'patm_ens' com valor -1 para todas as linhas
    df['patm_ens'] = -1  # Preenchido com -1 conforme solicitado

    # Certifique-se de que o diretório 'ascii_mgb' exista
    ascii_mgb_dir = os.path.join(my_dir, 'ascii_mgb')
    if not os.path.exists(ascii_mgb_dir):
        os.makedirs(ascii_mgb_dir)

    # Extrair o nome do arquivo sem o caminho e a extensão .csv
    name = os.path.splitext(os.path.basename(file))[0] + '.txt'

    # Caminho completo para o arquivo de saída no diretório 'ascii_mgb'
    output_path = os.path.join(ascii_mgb_dir, name)

    # Abrir o arquivo de saída em modo de escrita
    with open(output_path, "w") as text_file:
        text_file.write(df.to_string(col_space=col_space, justify='right', max_colwidth=16, index=False, header=False).replace("NaN", " -1"))

print("Arquivos .txt gerados com sucesso!")